In [1]:
import numpy as np
import itertools as it

import allel
import pandas as pd

from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

from datetime import datetime

import collections
def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

## MCM frequency evolution with changing Ne

### Changing theta in time

In this notebook we seek to model the evolution of allele frequencies when Ne changes in time.

Method: 

i) Define functions of Ne change in time.
- functions return Ne for generations.

ii) Integrate Ne.
- use scipy to integrate the changes in Ne forward in time.
- example: use integration to compare the expected number of mutations with and without Ne changes.

iii) Transition matrix.
- use matrix multiplication to calculate f(x) per generation. 
- function `single_gen_matrix_v2` is adapted to transition between population sizes.


### I. Ne in time functions.
define function that returns Ne as a function of generations. 

In [2]:
def theta_constant(tnow ,Ne= 1000):
    '''
    constant Ne
    '''
    if type(tnow).__module__ == np.__name__:
        Narray= np.ones(tnow.shape, dtype=int)
        Narray= Narray * Ne
        return Narray
    else:
        return Ne

def theta_exp(tnow, Ne= 1000, rate= 0.038):
    '''
    exponential growth or decline.
    '''
    if type(tnow).__module__ == np.__name__:
        
        Narray= Ne * np.exp(tnow * rate)
        return Narray
    
    else:
        Nt= Ne * np.exp(tnow * rate)
        return Nt

def theta_function(tnow, theta_time_array= [], inverse= False):
    '''
    pre-determined Ne scale. 
    '''
    if len(theta_time_array) == 0:
        print('no theta_time_array.')
        return tnow
    
    prime= np.where(theta_time_array[:,0] > tnow)[0]
    
    if len(prime) == 0:
        prime= theta_time_array[-1,1]
        
        if inverse:
            prime= 1 / prime
        
        return prime
    
    prime= theta_time_array[prime[0],1]
    
    if inverse:
        prime= 1 / prime
    
    return prime

In [3]:
# example using theta_function.

max_time= 4

theta_blocks= [2,6,1]
timez_blocks= [0.2,1.2,3]

time_array= np.array([
    timez_blocks,
    theta_blocks
]).T


###
###

t_func= theta_function

tfunc_kwargs= {
    'theta_time_array': time_array,
    'inverse':False
}

##
##

X_plot= np.linspace(0,max(timez_blocks) + 1,100)
y_plot= [t_func(x,**tfunc_kwargs) for x in X_plot]

fig_fn= [go.Scatter(
    x= X_plot,
    y= y_plot
)]


layout= go.Layout(
    title= 'select func of N over time, {}'.format(['N / Nt', 'Nt / N'][1-int(tfunc_kwargs['inverse'])]),
    yaxis= dict(range= [0,max(y_plot)+ 0.2]),
    height= 320
)

Figure= go.Figure(data= fig_fn, layout= layout)
iplot(Figure)

### II. Model parameters and integration. 

see [Simulations](https://nbviewer.jupyter.org/github/SantosJGND/Coalescent/blob/master/Simulations.ipynb) for examples of integration over Ne profiles. 

In [5]:
model_dict= {
    'test': {
        'T': 200,
        'Ne': 1000,
        'scale': 1
    }
}
pop_select= 'test'


### Number of mutations

In [6]:
pop_select= 'test'
scale_sim= False

muNuc= 1.08e-8
Ne= model_dict[pop_select]["Ne"]

if scale_sim:
    scale_gen= model_dict[pop_select]['scale']
    muNuc= muNuc / scale_gen
    Ne= int(Ne * scale_gen)

seqL= 1e6

mu= muNuc * seqL
Theta= 4 * Ne * mu

rate_Pmut= Theta / 2

print('Ne: {}'.format(Ne))
print('Theta: {}\nTotal mut rate: {}'.format(round(Theta,3),round(rate_Pmut,3)))

branch_len= model_dict[pop_select]['T']
MRCA= branch_len / Ne / 2

print("branch length: {}".format(MRCA))

Pexp= 2 * Ne * MRCA * Theta / 2 ## Multiply the standard poisson rate by 2Ne to do all the pop.


theta_dict= {
    'func':theta_exp,
    'kargs': {
        'rate': 0.0035
    }
}


Poisson_conv= theta_dict['func'](branch_len,Ne= Pexp,**theta_dict['kargs'])

print('poisson rate for uniform Ne= {} : {}'.format(Ne,Pexp))
print('poisson rate following integration: {}'.format(round(Poisson_conv,2)))


Ne: 1000
Theta: 43.2
Total mut rate: 21.6
branch length: 0.1
poisson rate for uniform Ne= 1000 : 4320.0
poisson rate following integration: 8699.41


In [7]:
muts= np.random.poisson(Poisson_conv,1)[0]

print('Nmuts: {}'.format(muts))

Nmuts: 8629


## MCM - Changing NE

functions to propagate gene frequences.


In [9]:
from tools.sfs_utilities import (
    single_gen_matrix_v2, freq_progr_func
)

#### Re-define model parameters

We are going to study f(x), the probability of an allele being found at frequency _x_, given a number `branch_len` of generations.

We set an initial Ne, which will serve as input parameter to the functions that map Ne to generation number. 

In [10]:
## And now we make it move
s= 0 # selection coefficient.
ploidy= 2 # ploidy.

branch_len= 200 # number of generations. 
Ne= 500 # Ne parameter.
fr= 1 # Initial frequency


#### Examples

i. Exponential

In [11]:
#####

rate_range= np.linspace(0.001,0.005,4)
exp_example= {}

for rate in rate_range:
    
    theta_dict= {
        'func':theta_exp,
        'kargs': {
            'rate': rate
        }
    }

    freq_ar, fixed_tally= freq_progr_func(theta_dict,Ne= Ne,T= branch_len,ploidy= ploidy, s= s)
    exp_example[rate]= freq_ar



ValueError: too many values to unpack (expected 2)

In [ ]:
fig= [go.Scatter(
    x= np.linspace(0,1,g.shape[1]),
    y= g[0],
    name= 'exp rate: '+ str(round(rate,5))
) for rate,g in exp_example.items()]

layout= go.Layout(
    title= 'Ngens: {}'.format(branch_len),
    xaxis= dict(
        title= 'frequency',
        range= [-0.05,1.05]
    ),
    yaxis= dict(
    title= 'f(x)'
    )
)

figure= go.Figure(data= fig,layout= layout)
iplot(figure)

ii. Uniform Ne.

In [ ]:
unif_example= {}
Ne_range= np.linspace(500,1000,4,dtype= int)

for Ne in Ne_range:
    
    theta_dict= {
        'func':theta_constant,
        'kargs': {}
    }
    
    freq_ar, fixed_tally= freq_progr_func(theta_dict,Ne= Ne,T= branch_len,ploidy= ploidy, s= s)
    unif_example[Ne]= freq_ar


In [ ]:
fig= [go.Scatter(
    x= np.linspace(0,1,g.shape[1]),
    y= g[0],
    name= 'Ne: ' + str(Ne)
) for Ne,g in unif_example.items()]

layout= go.Layout(
    title= 'Ngens: {}'.format(branch_len),
    xaxis= dict(
        title= 'frequency',
        range= [-0.05,1.05]
    ),
    yaxis= dict(
    title= 'f(x)'
    )
)
figure= go.Figure(data= fig,layout= layout)
iplot(figure)

In [ ]:
é